# Ingestion Explorator Analysis
## Summary at bottom

This project consists of large quantities of data sourced from the following sources
1. **nfl-data-py library:** A python library containing data related to NFL teams, players, schedules, stadiums and more.
    - Free and easy
    - Different sections of data (Player stats, injuries, schedules...) must be retrieved in their individual tables
    - Seemily efficient format
    
2. **The Odds API:** An API hosted [here](https://the-odds-api.com/) to retrieve betting odds and data relating to desired sports.
    - Free version or various pay up front plans based on request quantity.
        - Due to the amount of requests I'll need to make, I'm currently paying for the cheapest plan.
    - Game based bets or individual player props. For this project, I'll especially be focused on player props.
        - Player props are actually very new to their API. They are still improving them and will be adding more returned data.
            - I'm hoping that improvements include deduction in the amount of requests needed to get different props.

In [1]:
import os
import io
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
import pandas as pd

blob_service_client = BlobServiceClient.from_connection_string(os.getenv('NFL_STORAGE'))

# Container Clients
player_stats_client = blob_service_client.get_container_client('nfl-data-py')
odds_client = blob_service_client.get_container_client('odds-api')

# nfl-data-py Blob Clients
player_stats_blob_client = player_stats_client.get_blob_client('weekly-player-stats/2023_player_stats.csv')
player_injuries_blob_client = player_stats_client.get_blob_client('weekly-player-injuries/2023_player_injuries.csv')
schedules_blob_client = player_stats_client.get_blob_client('schedules/2023_schedule.csv')
rosters_blob_client = player_stats_client.get_blob_client('rosters/2023_roster.csv')

# odds-client
odds_blob_client = odds_client.get_blob_client('player-props/2023/NFL_data_2023-10-21-16-20-43.csv')

print('All Blob Clients successfully connected!')

All Blob Clients successfully connected!


In [2]:
# Due to the amount of tables, I'm using a function to take each blob client and convert it to a DataFrame
# Function created via chatGPT
def blob_to_df(blob_client):
    # Download the blob data
    download_stream = blob_client.download_blob()
    
    # Create an in-memory binary stream
    stream = io.BytesIO()
    download_stream.readinto(stream)
    stream.seek(0)  # Seek back to the start of the stream
    
    df = pd.read_csv(stream)
    return df

weekly_player_stats_df = blob_to_df(player_stats_blob_client)
weekly_player_injuries_df = blob_to_df(player_injuries_blob_client)
schedule_df = blob_to_df(schedules_blob_client)
roster_df = blob_to_df(rosters_blob_client)
odds_df = blob_to_df(odds_blob_client)

print('All data loaded into respective DataFrames!')

All data loaded into respective DataFrames!


## nfl-data-py data source
I am going to explore each relevant table from this source seprately and determine how they all tie together.

### Player Stats
This is the main table I am interested in, as it contains most of the relevant information. That being said, the other tables are also essential to piece everything together. The quantity of variables in this dataset is very high, so this exploratory data analysis (EDA) is essential. Lets start out by disyplaying a few rows of data.

In [3]:
pd.set_option('display.max_columns', None) #Force all columns to display
display(weekly_player_stats_df.head())

,player_id,player_name,player_display_name,position,position_group,recent_team,season,week,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,NYJ,2023,1,REG,BUF,0,1,0.0,0,0.0,1.0,10.0,0,0,17.0,0.0,0.0,-2.031960,0,0.000000,NaN,0,0.0,0,0.0,0.0,0.0,NaN,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.00,0.00
1,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,CHI,2023,4,REG,DEN,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,NaN,0,NaN,NaN,0,0.0,0,0.0,0.0,0.0,NaN,0,1,1,8.0,0,0.0,0.0,-3.0,11.0,0.0,0.483465,0,0.0,0.03125,-0.012397,0.038197,0.0,0.80,1.80
2,00-0026498,M.Stafford,Matthew Stafford,QB,QB,LA,2023,1,REG,SEA,24,38,334.0,0,0.0,0.0,-0.0,0,0,409.0,106.0,17.0,20.679981,0,0.816626,0.217190,3,11.0,0,0.0,0.0,1.0,0.868086,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,14.46,14.46
3,00-0026498,M.Stafford,Matthew Stafford,QB,QB,LA,2023,2,REG,SF,34,55,307.0,1,2.0,1.0,10.0,0,0,363.0,144.0,17.0,-5.089193,0,0.845730,-0.029705,4,17.0,0,0.0,0.0,1.0,-0.438330,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,13.98,13.98
4,00-0026498,M.Stafford,Matthew Stafford,QB,QB,LA,2023,3,REG,CIN,18,33,269.0,1,2.0,6.0,48.0,0,0,317.0,99.0,12.0,-8.404790,0,0.848580,-0.010766,1,7.0,0,0.0,0.0,1.0,2.529576,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,11.46,11.46


A lot of these columns are intuitive to those familiar with football. However, some of them are abbrieviations or acronyms that some people may be unfamilliar with. Fortunately, their is some documentation available [here](https://www.nflfastr.com/reference/calculate_player_stats.html). To shorten things up, I'll list some of the less intuitive columns below.

|Varaible|Definition|
|---|---|
|passing_yards|Yards gained from players pass plays (includes yards after catch)|
|passing_air_yards|Passing air yards (includes incomplete passes).|
|passing_epa|Total expected points added on pass attempts and sacks.|
|pacr|Passing Air Conversion Ratio. PACR = passing_yards / passing_air_yards|
|dakota|Adjusted EPA + CPOE composite based on coefficients which best predict adjusted EPA/play in the following year.|
|rushing_epa|Expected points added on rush attempts (incl. scrambles and kneel downs).|
|racr|Receiver Air Conversion Ratio. RACR = receiving_yards / receiving_air_yards|
|target_share||The share of targets of the player in all targets of his team|
|air_yards_share|The share of receiving_air_yards of the player in all air_yards of his team|
|wopr|Weighted Opportunity Rating. WOPR = 1.5 × target_share + 0.7 × air_yards_share|
|fantasy_points|Standard fantasy points.|
|fantasy_points_ppr|PPR (Point Per Reception) Fantasy Points|

It is important to understand that passing air yards includes incomplete passes, and not yards after catch. Let me simplify this with a single row from our DataFrame that is as simple as it gets.

In [4]:
display(weekly_player_stats_df.loc[(weekly_player_stats_df['player_id'] == '00-0028872') & (weekly_player_stats_df['week'] == 6) ])

,player_id,player_name,player_display_name,position,position_group,recent_team,season,week,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
24,00-0028872,J.Hekker,Johnny Hekker,P,SPEC,CAR,2023,6,REG,MIA,1,1,7.0,0,0.0,0.0,-0.0,0,0,-4.0,11.0,0.0,-1.941088,0,0.0,NaN,0,0.0,0,0.0,0.0,0.0,NaN,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.28,0.28


Looking at this, we can infer that it stats for a punter on special teams, so not someone who usually throws the ball. But, the Carolina Panthers ran a trick play that involved him throwing a single pass. You can see that he threw for 7 total passing_yards, despite having -4 passing_air_yards. This means, the reciever caught the ball 4 yards behind the line of scrimage, but still ran it 7 yards further than the line of scrimage. This concept can be reinforced by inspecting the 'passing_yards_after_catch' variable being equal to 11. If the reciever had not caught the ball, passing_air_yards would still be -4, but passing_yards would be 0 since it only includes completions.

#### Dtypes
I checked the dtypes, but due to the amount of columns will not display them. In esscence, all the obvious strings are of type object, and then every other number that could be interpreted as a number, is either int or float. This will make calculations and comparisons much easier to work with. It is improtant to understand that the player_id is actually of type object, due to the hyphens often contained within.

### Player Injuries

This table will be interesting to see how a player dealing with injury in the days before a game effect their weekly stats. It should help predict whether or not a player is going to be significantly hindered due to a certain injury or even illness.

In [5]:
weekly_player_injuries_df.head()

,season,game_type,team,week,gsis_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified
0,2023,REG,ARI,1,00-0034473,LB,Dennis Gardeck,Dennis,Gardeck,NaN,NaN,NaN,Knee,NaN,Full Participation in Practice,2023-09-08 18:49:43+00:00
1,2023,REG,ARI,1,00-0029316,T,Kelvin Beachum,Kelvin,Beachum,Hand,NaN,Out,Hand,NaN,Did Not Participate In Practice,2023-09-08 18:50:25+00:00
2,2023,REG,ARI,1,00-0035662,WR,Marquise Brown,Marquise,Brown,Hamstring,NaN,Questionable,Hamstring,NaN,Limited Participation in Practice,2023-09-08 18:50:02+00:00
3,2023,REG,ARI,1,00-0030061,TE,Zach Ertz,Zach,Ertz,Knee,NaN,Questionable,Knee,NaN,Limited Participation in Practice,2023-09-08 18:50:02+00:00
4,2023,REG,ATL,1,00-0033858,TE,Jonnu Smith,Jonnu,Smith,NaN,NaN,NaN,Not injury related - resting player,NaN,Did Not Participate In Practice,2023-09-06 20:54:47+00:00


This table is dynamic with a 'date_modified' column so we can see how a players injury is potentially improving over time. We can view the practice_primary_injury column to see what the injury is, and then the 'practice_column' variable to see if it effected their practice. We are also able to see if a player missed practice due to a non-injury, such as needing rest or being ill. You can see that the report_primary_injury column is not always filled when the practice_primary_injury is. This is because a player's injury can improve to the point that they don't get listed on the finaly injury report, that's due before the game starts. 

'gsis_id' is the unique player id, and ties to 'player_id' in the player stats table allowing for easy merging of data, especially when including season and week in the merging criteria.

#### dtypes

As expected, almost every column is of type object. The only two varables that arent, are 'season' and 'week', which are of type int.

### Schedule 

Schedule will be useful primarily to see upcoming mathcups, allowing to more accurately form expectations of players based on who they will be facing. All games are included, but to get an understanding of the format, I'm going to look at just the Minnesota Vikings schedule.

In [6]:
schedule_df.loc[(schedule_df['home_team'] == 'MIN') | (schedule_df['away_team'] == 'MIN')]

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,div_game,location,result,total,overtime,gsis,nfl_detail_id,temp,wind,stadium_id
5,2023_01_TB_MIN,2023,REG,1,2023-09-10,Sunday,13:00,TB,20.0,MIN,17.0,0,Home,-3.0,37.0,0.0,NaN,NaN,NaN,NaN,MIN01
16,2023_02_MIN_PHI,2023,REG,2,2023-09-14,Thursday,20:15,MIN,28.0,PHI,34.0,0,Home,6.0,62.0,0.0,NaN,NaN,NaN,NaN,PHI00
39,2023_03_LAC_MIN,2023,REG,3,2023-09-24,Sunday,13:00,LAC,28.0,MIN,24.0,0,Home,-4.0,52.0,0.0,NaN,NaN,NaN,NaN,MIN01
51,2023_04_MIN_CAR,2023,REG,4,2023-10-01,Sunday,13:00,MIN,21.0,CAR,13.0,0,Home,-8.0,34.0,0.0,NaN,NaN,78.0,7.0,CAR00
75,2023_05_KC_MIN,2023,REG,5,2023-10-08,Sunday,16:25,KC,27.0,MIN,20.0,0,Home,-7.0,47.0,0.0,NaN,NaN,NaN,NaN,MIN01
81,2023_06_MIN_CHI,2023,REG,6,2023-10-15,Sunday,13:00,MIN,19.0,CHI,13.0,1,Home,-6.0,32.0,0.0,NaN,NaN,53.0,16.0,CHI98
105,2023_07_SF_MIN,2023,REG,7,2023-10-23,Monday,20:15,SF,NaN,MIN,NaN,0,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN01
109,2023_08_MIN_GB,2023,REG,8,2023-10-29,Sunday,13:00,MIN,NaN,GB,NaN,1,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GNB00
124,2023_09_MIN_ATL,2023,REG,9,2023-11-05,Sunday,13:00,MIN,NaN,ATL,NaN,0,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL97
141,2023_10_NO_MIN,2023,REG,10,2023-11-12,Sunday,13:00,NO,NaN,MIN,NaN,0,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIN01


We see that we have a game_id as a unique game identifier. Or, if we wanted to get all games for a certain week, we could use the combination of season and week. We also have a stadium_id representing the stadium the game is being played at. This ties to an id in a seperate stadium table, that is not yet incorporated in my project.

While we have the entire season schedule, we have additional information for completed games. This includes the results, weather, and more. This table is dynamic to reflect not just that additional information, but also any schedule changes.

#### dtypes
The dtypes here are all about what you'd expect, either object, int or float. One change that should possible be made is changing the gameday variable from type object to datetime.

### Roster

We have dynamic rosters for every team in the league. I'm going to analyze some of the vikings roster. Due to the number of players on each team, I will only display a small fraction of their available roster.

In [7]:
roster_df.loc[roster_df['team'] == 'MIN'].head()

,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age
62,2023,MIN,QB,QB,8.0,ACT,Kirk Cousins,Kirk,Cousins,1988-08-19,75.0,202,Michigan State,00-0029604,14880.0,bbd0942c-6f77-4f83-a6d0-66ec6548019e,25812.0,8057.0,7102.0,CousKi00,14252.0,1166.0,11,NaN,7,REG,A01,Kirk,COU709400,38632,3200434f-5570-9400-e1ae-f835abb5963e,2012,2012.0,WAS,102.0,35.0
63,2023,MIN,DB,SS,22.0,ACT,Harrison Smith,Harrison,Smith,1989-02-02,74.0,214,Notre Dame,00-0029606,14945.0,407f1923-6659-4564-800f-25b8746d6d3e,25739.0,8202.0,7641.0,SmitHa00,14461.0,1213.0,11,NaN,7,REG,A01,Harrison,SMI317465,38559,3200534d-4931-7465-ebfe-5c05b57e543c,2012,2012.0,MIN,29.0,34.0
72,2023,MIN,LS,LS,42.0,ACT,Andrew DePaola,Andrew,DePaola,1987-07-28,73.0,230,Rutgers,00-0029714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,NaN,7,REG,A01,Andrew,DEP118180,39750,32004445-5011-8180-19a1-6814db083a3b,2012,2012.0,NaN,NaN,36.0
89,2023,MIN,OL,G,76.0,ACT,David Quessenberry,David,Quessenberry,1990-08-24,77.0,310,San Jose State,00-0030097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,7,REG,A01,David,QUE597984,40124,32005155-4559-7984-fe6c-8f632e818cdc,2013,2013.0,HOU,176.0,33.0
182,2023,MIN,LB,OLB,99.0,ACT,Danielle Hunter,Danielle,Hunter,1994-10-29,77.0,252,Louisiana State,00-0031565,2976560.0,ba7fe857-df63-4aed-803a-80993b157be4,28476.0,10318.0,9521.0,HuntDa01,16849.0,2393.0,8,NaN,7,REG,A01,Danielle,HUN529240,42431,32004855-4e52-9240-4ef3-07ebfed3439a,2015,2015.0,MIN,88.0,28.0


We have a surplus of variables available for every player on the roster. A lot of them are unecessary for my business goal. We once again have a player_id that maps to our player_id in the weekly stats table. We additionally have id's that various other sources use for the player. None of these seem important to me as of now, but it's useful to know that it's available.

I anticipate this table not getting used much, outside of storing general information on players and rosters, if ever needed to be referenced.

#### dtypes
The dtypes are float when possible, otherwise object. All the ID's vary, as some contain characters, others don't.

## Odds Data Source

I will be collecting data from this source often, but as I've been setting up the ingestion process, data consumption was pretty infrequent. The dataframe I'm working with contains most of the data I have up to this point, which represents less than 5 different runs of my ingestion script. In the very near future, it will be running it more than that within a single week.

Despite that, the amount of data returned is of great quantity, but the number of variables is relatively small. I am going to inspect a random sample below, since the actual table is over 17,000 rows already.

In [26]:
odds_df.sample(10)

,Player Name,Event ID,Commence Time,Away Team,Home Team,Bookmaker Key,Bookmaker Title,Market Key,Outcome Name,Price,Point,Refresh Time
12842,Romeo Doubs,e5a039468378fe43795987ade2d44536,2023-10-22 16:26:00-04:00,Green Bay Packers,Denver Broncos,betrivers,BetRivers,player_reception_yds,Under,1.87,39.5,2023-10-21T17:20:41.713422-04:00
2204,Lamar Jackson,17d4277bfa01c37a1c599fb1fb3d3d13,2023-10-22 13:01:00-04:00,Detroit Lions,Baltimore Ravens,bovada,Bovada,player_pass_yds,Under,1.87,215.5,2023-10-21T17:20:38.057124-04:00
2664,Kerby Joseph,17d4277bfa01c37a1c599fb1fb3d3d13,2023-10-22 13:01:00-04:00,Detroit Lions,Baltimore Ravens,betrivers,BetRivers,player_tackles_assists,Under,1.76,5.5,2023-10-21T17:20:38.057124-04:00
3556,Josh Allen,3080378924a7a8fa778d90243a1f4d54,2023-10-22 13:01:00-04:00,Buffalo Bills,New England Patriots,williamhill_us,Caesars,player_pass_interceptions,Over,1.74,0.5,2023-10-21T17:20:38.566702-04:00
336,Desmond Ridder,949fd4373890ee339add53eb5f343dc0,2023-10-22 13:01:00-04:00,Atlanta Falcons,Tampa Bay Buccaneers,draftkings,DraftKings,player_rush_attempts,Under,1.57,3.5,2023-10-21T17:20:37.495415-04:00
7740,Tyrod Taylor,50bd8049327bddb2433b90e98994d98b,2023-10-22 13:01:00-04:00,Washington Commanders,New York Giants,bovada,Bovada,player_anytime_td,Yes,5.00,NaN,2023-10-21T17:20:40.142021-04:00
7965,Jalin Hyatt,50bd8049327bddb2433b90e98994d98b,2023-10-22 13:01:00-04:00,Washington Commanders,New York Giants,unibet_us,Unibet,player_receptions,Over,1.63,1.5,2023-10-21T17:20:40.142021-04:00
4955,DeMarcus Walker,2decc44ccdfb309756fa93d864f5773f,2023-10-22 13:01:00-04:00,Las Vegas Raiders,Chicago Bears,williamhill_us,Caesars,player_anytime_td,Yes,101.00,NaN,2023-10-21T17:20:39.101299-04:00
14318,Noah Gray,c992a4e4d0046cd4926bf3524e35ff79,2023-10-22 16:26:00-04:00,Los Angeles Chargers,Kansas City Chiefs,unibet_us,Unibet,player_reception_yds,Under,1.88,14.5,2023-10-21T17:20:42.238171-04:00
8398,Graham Gano,50bd8049327bddb2433b90e98994d98b,2023-10-22 13:01:00-04:00,Washington Commanders,New York Giants,barstool,Barstool Sportsbook,player_field_goals,Under,1.67,1.5,2023-10-21T17:20:40.142021-04:00


As you can see, their are only 12 variables. Information on the players, game times, and teams are intuitive. Some columns will not be intuitive to someone newer to sports betting, so I will give their descriptions below:

|Variable|Definition|
|---|---|
|Bookmaker Key|Represents the provider of the offered bet|
|Market Key|Specifies what kind of bet is being offerred. Could be player pass yards, recieving yards, touchdowns, or so much more|
|Point|Specifies the line associated with the bet. The point could be 4.5 receptions, 249.5 passing yards, or whatever else|
|Outcome Name|Needs to be considered in association with the Point. Indicatsd if the bet is for the player going 'Over' or 'Under' the Point. Note: can also be 'Yes' or 'No' for certain bets, such as a player scoring the first touchdown|
|Price|Multiplier applied to your bet if you win. A 5 dollar bet win on a 2.0 multiplier would return 10 dollars|


#### dtypes

All variables except for price and point are of type object. Price and point are of type float. 

I mentioned that the player props are a newer addition to their API, and still being improved. The biggest issue I see, is not being able to uniquely identify palyers if they have the same name as someone else, since their is no player_id. I contacted the API team and asked about plans to add this, or even a player position variable and was informed they have plans coming to introduce a way to identify players, but their actual approach is not decided yet. Until then, I will need to figure my own approach out, that can also tie with my other data source. I anticipate this being one of my biggest challenges.

# Summary


### nfl-data-py
The nfl-data-py data source is for the most part very, ideal. The quantity of information is not too much, although once we want to use many years of data, the weekly player stats will grow in size fairly drastically. The other files should remain very manageable, due to their natural overwrite / update system enforced by the data source. 

This format of this source looks nice, with variables being of very reasonable types (numeric when possible). All the data ties together well, and I'm looking forward to building up my project with this source.

### Odds API
This data source is still relatively new, and actively being improved. Until then, their are some aspects that seem less than ideal, such as no player_id or position being returned by the API. This makes it seem nearly impossible to identify a unique player if they share a name with someone else. They have plans to fix this in the near future, but the ETA is unspecified. I hope to see them also decrease the number of requests needed to get all the props, because right now it requires a lot of them to get all the different types of bets, and I'm having to pay for the cheapest version of the API, versus taking advantage of the free version.

The size of this data source will grow very large, and I will need to determine the best way to injest and store it all.

### Challenges
The main challenges I see are:
- trying to uniquely identify players from the Odds API and tie it together with the rest of the data from the other data source.
- Storing the large amount of data from the Odds API.
- Making sure all the tables from nfl-data-py are efficiently tied together.